# HDSC2022 Capstone Project
## by Team Prophet

The dataset contains information about the scholarship programs in China as of May 2019. <br>
The data was collected through by scraping the [CUCAS website](https://www.cucas.edu.cn/china_scholarships/). <br>
The code to the web scraping program and data cleaning program can be found [here](https://github.com/mcmuralishclint/CUCAS). <br>

The goal of this project is to ... .

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, OrdinalEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
# We are going to read the csv file using read_excel() function 
data = pd.read_excel('./data/china_scholarship.xls')  #might also need to pip install xlrd
#data is our upgraded dataset

In [3]:
data.head()

,schol_id,University,Major,Location / District,Province / City,Level,Language,Tuition Covered,Accomodation covered?,Living Expense Covered?,Tuition fees to pay,Original Tuition fee,Start_Month,Start_Year,Accomodation_To_Pay,Accomodation_duration,Expense_To_Pay,Expense_duration
0,1,Shanghai University of Traditional Chinese Med...,Summer School (TCM),Xuhui,Shanghai,Non-Degree,English,7500.0,0,0,0,7500.0,Jun,2020,9750,MONTH,1000,MONTH
1,2,Changchun University of Science and Technology,Preparatory Program of Humanities Direction(Pa...,Chaoyang,Jilin,Non-Degree,Chinese,10000.0,0,0,10000,20000.0,Sep,2019,1500,MONTH,1500,MONTH
2,3,Changchun University of Science and Technology,Preparatory Program of Business Direction(Path A),Chaoyang,Jilin,Non-Degree,Chinese,10000.0,0,0,10000,20000.0,Sep,2019,1500,MONTH,1500,MONTH
3,4,Changchun University of Science and Technology,Preparatory Program of Science and Engineering...,Chaoyang,Jilin,Non-Degree,Chinese,10000.0,0,0,10000,20000.0,Sep,2019,1500,MONTH,1500,MONTH
4,5,Qingdao University,Long-term Chinese Language Program,Qingdao,Shandong,Non-Degree,Chinese,7000.0,1,1,0,7000.0,Sep,2019,0,YEAR,0,MONTH


In [4]:
data.columns

Index(['schol_id', 'University', 'Major', 'Location / District',
       'Province / City', 'Level', 'Language', 'Tuition Covered',
       'Accomodation covered?', 'Living Expense Covered?',
       'Tuition fees to pay', 'Original Tuition fee', 'Start_Month',
       'Start_Year', 'Accomodation_To_Pay', 'Accomodation_duration',
       'Expense_To_Pay', 'Expense_duration'],
      dtype='object')

In [5]:
# We are going to rename the columns using the snake_case naming convention to improve readability
data.columns = ['school_id', 'university', 'major', 'district', 'city', 'level', 'language', 'tuition_covered', 'accomodation_covered',
'living_expense_covered', 'tuition_fees_to_pay', 'original_tuition_fee','start_month','start_year', 'accomodation_to_pay',
'accomodation_duration', 'expense_to_pay', 'expense_duration']

In [6]:
# Let us view an intuitive summary of our dataframe
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3576 entries, 0 to 3575
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   school_id               3576 non-null   int64  
 1   university              3576 non-null   object 
 2   major                   3576 non-null   object 
 3   district                3576 non-null   object 
 4   city                    3576 non-null   object 
 5   level                   3576 non-null   object 
 6   language                3576 non-null   object 
 7   tuition_covered         3453 non-null   float64
 8   accomodation_covered    3576 non-null   int64  
 9   living_expense_covered  3576 non-null   int64  
 10  tuition_fees_to_pay     3576 non-null   int64  
 11  original_tuition_fee    3453 non-null   float64
 12  start_month             3576 non-null   object 
 13  start_year              3576 non-null   int64  
 14  accomodation_to_pay     3576 non-null   

In [7]:
# Now let us see a preview of the data by looking at the first 5 rows 
data.head()

,school_id,university,major,district,city,level,language,tuition_covered,accomodation_covered,living_expense_covered,tuition_fees_to_pay,original_tuition_fee,start_month,start_year,accomodation_to_pay,accomodation_duration,expense_to_pay,expense_duration
0,1,Shanghai University of Traditional Chinese Med...,Summer School (TCM),Xuhui,Shanghai,Non-Degree,English,7500.0,0,0,0,7500.0,Jun,2020,9750,MONTH,1000,MONTH
1,2,Changchun University of Science and Technology,Preparatory Program of Humanities Direction(Pa...,Chaoyang,Jilin,Non-Degree,Chinese,10000.0,0,0,10000,20000.0,Sep,2019,1500,MONTH,1500,MONTH
2,3,Changchun University of Science and Technology,Preparatory Program of Business Direction(Path A),Chaoyang,Jilin,Non-Degree,Chinese,10000.0,0,0,10000,20000.0,Sep,2019,1500,MONTH,1500,MONTH
3,4,Changchun University of Science and Technology,Preparatory Program of Science and Engineering...,Chaoyang,Jilin,Non-Degree,Chinese,10000.0,0,0,10000,20000.0,Sep,2019,1500,MONTH,1500,MONTH
4,5,Qingdao University,Long-term Chinese Language Program,Qingdao,Shandong,Non-Degree,Chinese,7000.0,1,1,0,7000.0,Sep,2019,0,YEAR,0,MONTH


In [8]:
# Let us also look at the last 5 rows of the dataframe
data.tail()

,school_id,university,major,district,city,level,language,tuition_covered,accomodation_covered,living_expense_covered,tuition_fees_to_pay,original_tuition_fee,start_month,start_year,accomodation_to_pay,accomodation_duration,expense_to_pay,expense_duration
3571,3601,East China University of Science and Technology,Environmental Science and Engineering,Xuhui District,Shanghai,Phd,English,38950.0,1,1,0,38950.0,Sep,2019,0,YEAR,0,MONTH
3572,3602,East China University of Science and Technology,Power Engineering and Engineering Thermophysics,Xuhui District,Shanghai,Phd,English,38950.0,1,1,0,38950.0,Sep,2019,0,YEAR,0,MONTH
3573,3603,East China University of Science and Technology,Thermal Engineering,Xuhui District,Shanghai,Phd,English,38950.0,1,1,0,38950.0,Sep,2019,0,YEAR,0,MONTH
3574,3604,East China University of Science and Technology,Synthesis of Fine Chemical and Molecular,Xuhui District,Shanghai,Phd,English,38950.0,1,1,0,38950.0,Sep,2019,0,YEAR,0,MONTH
3575,3605,East China University of Science and Technology,Sociology of Law,Xuhui District,Shanghai,Phd,English,38950.0,1,1,0,38950.0,Sep,2019,0,YEAR,0,MONTH


In [9]:
# Let us see how many null values present in the columns 
data.isna().sum()

school_id                   0
university                  0
major                       0
district                    0
city                        0
level                       0
language                    0
tuition_covered           123
accomodation_covered        0
living_expense_covered      0
tuition_fees_to_pay         0
original_tuition_fee      123
start_month                 0
start_year                  0
accomodation_to_pay         0
accomodation_duration       0
expense_to_pay              0
expense_duration            0
dtype: int64

In [10]:
# Let us view the descriptive statistics of the dataframe. Note that this can only be applied to numerical values 
data[['tuition_covered', 'tuition_fees_to_pay', 'original_tuition_fee', 'accomodation_to_pay', 'expense_to_pay']].describe().T

,count,mean,std,min,25%,50%,75%,max
tuition_covered,3453.0,20531.704315,12428.124918,0.0,10625.0,20000.0,26500.0,80000.0
tuition_fees_to_pay,3576.0,8433.516499,9519.340675,0.0,0.0,6000.0,11000.0,69000.0
original_tuition_fee,3453.0,28693.955980,15341.450293,0.0,20000.0,26500.0,34000.0,148000.0
accomodation_to_pay,3576.0,1376.461969,2802.526970,0.0,400.0,900.0,1200.0,24000.0
expense_to_pay,3576.0,1070.987136,601.297528,0.0,1000.0,1000.0,1500.0,2700.0


## Data Preprocessing

In [11]:
temp = data[['original_tuition_fee', 'tuition_covered', 'tuition_fees_to_pay']]
temp["check"] = data['original_tuition_fee'] - data['tuition_covered']
temp

,original_tuition_fee,tuition_covered,tuition_fees_to_pay,check
0,7500.0,7500.0,0,0.0
1,20000.0,10000.0,10000,10000.0
2,20000.0,10000.0,10000,10000.0
3,20000.0,10000.0,10000,10000.0
4,7000.0,7000.0,0,0.0
...,...,...,...,...
3571,38950.0,38950.0,0,0.0
3572,38950.0,38950.0,0,0.0
3573,38950.0,38950.0,0,0.0
3574,38950.0,38950.0,0,0.0


In [12]:
print((temp['tuition_fees_to_pay'] != temp['check']).sum())

123


In [13]:
temp[temp['tuition_fees_to_pay'] != temp['check']]

,original_tuition_fee,tuition_covered,tuition_fees_to_pay,check
883,NaN,NaN,15000,NaN
1029,NaN,NaN,15000,NaN
2395,NaN,NaN,16200,NaN
2396,NaN,NaN,16200,NaN
2397,NaN,NaN,16200,NaN
...,...,...,...,...
3260,NaN,NaN,16200,NaN
3261,NaN,NaN,16200,NaN
3262,NaN,NaN,0,NaN
3320,NaN,NaN,16200,NaN


### Tuition to pay
There are only 123 instances where the difference between orignal tuition and tuition covered is not equal to tuition to pay. <br> and this number 123 corresponds to the number of missing values  for both tuition covered and original tuition. <br>
Based on the above, tuition to pay is not a good target for prediction.

In [60]:
data["accomodation_duration"].value_counts()

MONTH       2575
YEAR         774
DAY          116
TERM          59
SEMESTER      27
-1            25
Name: accomodation_duration, dtype: int64

In [61]:
newdata = data.copy()

#convert per year to per month
#newdata['accomodation_to_pay'][newdata['accomodation_duration'] == "YEAR"] = newdata['accomodation_to_pay'][newdata['accomodation_duration'] == "YEAR"] / 12

#convert per day to per year using 365 days per year
newdata['accomodation_to_pay'][newdata['accomodation_duration'] == "DAY"] = newdata['accomodation_to_pay'][newdata['accomodation_duration'] == "DAY"] * 365

#convert per term to per year using 3 terms in a year
newdata['accomodation_to_pay'][newdata['accomodation_duration'] == "TERM"] = newdata['accomodation_to_pay'][newdata['accomodation_duration'] == "TERM"] * 2

#convert per semester to per year using 2 semesters per year
newdata['accomodation_to_pay'][newdata['accomodation_duration'] == "SEMESTER"] = newdata['accomodation_to_pay'][newdata['accomodation_duration'] == "SEMESTER"] * 2

newdata.sample(5)

,school_id,university,major,district,city,level,language,tuition_covered,accomodation_covered,living_expense_covered,tuition_fees_to_pay,original_tuition_fee,start_month,start_year,accomodation_to_pay,accomodation_duration,expense_to_pay,expense_duration
534,564,South China University of Technology,Law,Tianhe,Guangdong,Bachelor,Chinese,17550.0,0,0,6500,24050.0,Sep,2019,1000,MONTH,1000,MONTH
1910,1940,Ningbo University,English Linguistics & Literature,Jiangbei,Zhejiang,Master,English,5000.0,0,0,21500,26500.0,Sep,2019,120,MONTH,1000,MONTH
2400,2430,Shandong University,Nutrition and Food Hygiene,Licheng District,Shandong,Master,English,NaN,0,1,16200,NaN,Sep,2019,810,MONTH,0,MONTH
3418,3448,North China Electric Power University,Radiochemistry,Changping,Beijing,Phd,Chinese,0.0,0,1,35000,35000.0,Sep,2019,1200,MONTH,0,MONTH
1207,1237,China University of Mining and Technology,Human Resource Management,Xuzhou,Jiangsu,Bachelor,Chinese,5375.0,0,0,9025,14400.0,Sep,2019,3000,YEAR,2000,MONTH


In [62]:
temp2 = data[['living_expense_covered', 'expense_to_pay', "expense_duration"]]
temp2

,living_expense_covered,expense_to_pay,expense_duration
0,0,1000,MONTH
1,0,1500,MONTH
2,0,1500,MONTH
3,0,1500,MONTH
4,1,0,MONTH
...,...,...,...
3571,1,0,MONTH
3572,1,0,MONTH
3573,1,0,MONTH
3574,1,0,MONTH


In [63]:
data["expense_duration"].value_counts()

MONTH    3576
Name: expense_duration, dtype: int64

### Accomodation to pay
Accomodation to pay is always zero when accomodation is covered.

### Expense to pay
Expense to pay is zero when living expenses are covered. <br>
Drop expense_duration as it is always per month, there's nothing to learn from it.

In [64]:
#Create total expense to pay, sum of tuition to pay, accomodation to pay and expense to pay.
newdata["total_expense"] = newdata[["tuition_fees_to_pay", "accomodation_to_pay", "expense_to_pay"]].sum(axis=1)
newdata.sample(5)

,school_id,university,major,district,city,level,language,tuition_covered,accomodation_covered,living_expense_covered,tuition_fees_to_pay,original_tuition_fee,start_month,start_year,accomodation_to_pay,accomodation_duration,expense_to_pay,expense_duration,total_expense
3456,3486,China University of Petroleum - Beijing,Marine Oil & Gas Engineering,Changping,Beijing,Phd,Chinese,36000.0,1,1,0,36000.0,Sep,2019,750,MONTH,750,MONTH,1500
2052,2082,Northeast Petroleum University,Chemical engineering and technology,Longfeng,Heilongjiang,Master,Chinese,7500.0,0,0,14500,22000.0,Sep,2019,1000,MONTH,1000,MONTH,16500
3208,3238,Shandong University,Quantitative Economics,Licheng District,Shandong,Phd,English,25000.0,0,0,6000,31000.0,Sep,2019,810,MONTH,1500,MONTH,8310
2613,2643,North China Electric Power University,Public policy (energy policy),Changping,Beijing,Master,Chinese,0.0,0,1,30000,30000.0,Sep,2019,1200,MONTH,0,MONTH,31200
1405,1435,Beijing Foreign Studies University-Internation...,E-commerce(Beijing Campus),Haidian,Beijing,Bachelor,English,37400.0,0,0,9950,47350.0,Sep,2019,1200,MONTH,900,MONTH,12050


In [65]:
newdata.columns

Index(['school_id', 'university', 'major', 'district', 'city', 'level',
       'language', 'tuition_covered', 'accomodation_covered',
       'living_expense_covered', 'tuition_fees_to_pay', 'original_tuition_fee',
       'start_month', 'start_year', 'accomodation_to_pay',
       'accomodation_duration', 'expense_to_pay', 'expense_duration',
       'total_expense'],
      dtype='object')

In [66]:
#remove redundant columns
redund_cols = ['tuition_fees_to_pay', 'original_tuition_fee',
               'accomodation_to_pay', 'tuition_covered',
               'accomodation_duration', 'expense_to_pay', 'expense_duration',
              "school_id", "major", "city"] #'tuition_covered', add/remove tuition covered to/from the features
newdata = newdata.drop(columns=redund_cols)

In [67]:
newdata

,university,district,level,language,accomodation_covered,living_expense_covered,start_month,start_year,total_expense
0,Shanghai University of Traditional Chinese Med...,Xuhui,Non-Degree,English,0,0,Jun,2020,10750
1,Changchun University of Science and Technology,Chaoyang,Non-Degree,Chinese,0,0,Sep,2019,13000
2,Changchun University of Science and Technology,Chaoyang,Non-Degree,Chinese,0,0,Sep,2019,13000
3,Changchun University of Science and Technology,Chaoyang,Non-Degree,Chinese,0,0,Sep,2019,13000
4,Qingdao University,Qingdao,Non-Degree,Chinese,1,1,Sep,2019,0
...,...,...,...,...,...,...,...,...,...
3571,East China University of Science and Technology,Xuhui District,Phd,English,1,1,Sep,2019,0
3572,East China University of Science and Technology,Xuhui District,Phd,English,1,1,Sep,2019,0
3573,East China University of Science and Technology,Xuhui District,Phd,English,1,1,Sep,2019,0
3574,East China University of Science and Technology,Xuhui District,Phd,English,1,1,Sep,2019,0


### Inspect columns in new data

In [68]:
newdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3576 entries, 0 to 3575
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   university              3576 non-null   object
 1   district                3576 non-null   object
 2   level                   3576 non-null   object
 3   language                3576 non-null   object
 4   accomodation_covered    3576 non-null   int64 
 5   living_expense_covered  3576 non-null   int64 
 6   start_month             3576 non-null   object
 7   start_year              3576 non-null   int64 
 8   total_expense           3576 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 251.6+ KB


In [69]:
newdata["district"].nunique()

40

In [70]:
newdata["district"].unique()

array(['Xuhui', 'Chaoyang ', 'Qingdao', 'Zhongyuan', 'Xicheng Qu',
       'Jiangbei ', 'Shinan', 'Changping', 'Haidian', "Lin'an", 'Nankai',
       'Nanchang', 'Xinbei', "Chang'an", 'Wuhan', 'Licheng', 'Wucheng',
       'Chaoyang', 'Fuchen', 'Tiexi', 'Dongxihu', 'Tianhe', 'Zhenjiang',
       'Xihu', 'Shahekou', 'Yangpu', 'Longfeng', 'Yunlong', 'Huanggu',
       'Xuzhou', 'Nanjing', 'Hexi', 'Siping', ' Fengxian', 'Jinping',
       'Dongguan', 'Yuzhong', 'Baoji', 'Licheng District',
       'Xuhui District'], dtype=object)

In [71]:
newdata = newdata.replace("Licheng District", "Licheng")
newdata = newdata.replace("Xuhui District", "Xuhui")

In [72]:
newdata["district"].nunique()

38

There are 38 unique disricts, we could predict total expense based on district.

In [73]:
newdata["level"].unique()

array(['Non-Degree', 'Bachelor', 'Master', 'Phd'], dtype=object)

In [74]:
newdata["university"].nunique()

53

In [75]:
newdata["language"].unique()

array(['English', 'Chinese', 'Japanese', 'German'], dtype=object)

In [76]:
newdata["start_month"].nunique()

5

In [77]:
newdata["start_year"].nunique()

2

## Modelling

In [78]:
# Let us define the predictor columns and the target column 
# X is the predictor
# y is the target variable
X = newdata.drop(['total_expense'], axis = 1)
y = newdata['total_expense']

In [79]:
# Here we are differentiating between the columns that are objects and the ones that are not
obj_cols = list(X.select_dtypes(include = 'object').columns)
num_cols = list(X.select_dtypes(exclude = 'object').columns)
print(obj_cols)
print(num_cols)

['university', 'district', 'level', 'language', 'start_month']
['accomodation_covered', 'living_expense_covered', 'start_year']


In [80]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3576 entries, 0 to 3575
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   university              3576 non-null   object
 1   district                3576 non-null   object
 2   level                   3576 non-null   object
 3   language                3576 non-null   object
 4   accomodation_covered    3576 non-null   int64 
 5   living_expense_covered  3576 non-null   int64 
 6   start_month             3576 non-null   object
 7   start_year              3576 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 223.6+ KB


In [81]:
#Split into train and validation
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                      test_size=0.3,
                                                    random_state=42)

print(f"Train size: \nxtrain: {X_train.shape}\nytrain: {y_train.shape}",
      f"\n\nTest size: \nxtest: {X_test.shape}\nytest: {y_test.shape}")

Train size: 
xtrain: (2503, 8)
ytrain: (2503,) 

Test size: 
xtest: (1073, 8)
ytest: (1073,)


In [82]:
# Preprocessing for numerical data
#data is filled with the mean value
#data is scaled with MinMaxScaler
numerical_transformer = Pipeline(steps=[("impute", SimpleImputer(strategy="mean")), 
                                        ('scale', MinMaxScaler())])

# Preprocessing for categorical data
#data is encoded with OrdinalEncoder
#data is scaled with MinMaxScaler
categorical_transformer = Pipeline(steps=[
    ('label_enc', OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=100)), ('scale', MinMaxScaler())])  
#Changed to Ordinal Encoder to reduce dimensions

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[('num', numerical_transformer, num_cols),
        ('cat', categorical_transformer, obj_cols)])

## Random Forest

In [83]:
model = RandomForestRegressor()

# Bundle preprocessing and modeling code in a pipeline
rf_clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model) ])

scores = cross_val_score(rf_clf, X_train, y_train, cv=5,
                        scoring='neg_mean_absolute_error')
print("MAE scores:\n", scores)
print(scores.mean())

MAE scores:
 [ -933.70990915 -1022.17807626 -1320.37156896  -930.03344177
  -782.20144846]
-997.6988889195634


In [84]:
rf_clf.fit(X_train, y_train)
rf_clf.score(X_test, y_test)

0.9329107754115356

In [85]:
def rf_score(params):
    model = RandomForestRegressor(n_jobs=-1, **params)
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)])
    mae = -1 * cross_val_score(pipe, X_train, y_train, cv=5,
                        scoring='neg_mean_absolute_error').mean()
    return mae


def rf_objective(trial):
    params = {
                "criterion" : trial.suggest_categorical("criterion", ["mse", "mae"]),
                "n_estimators" : trial.suggest_int('n_estimators', 1, 1000),
                'max_depth':trial.suggest_int('max_depth', 1, 7),
             }
    return(rf_score(params))


def print_best_callback(study, trial):
    print(f"Best value: {study.best_value}")

In [86]:
optuna.logging.set_verbosity(0)
rf_study = optuna.create_study(direction='minimize',sampler=TPESampler())
rf_study.optimize(rf_objective, n_trials= 20, show_progress_bar = True, callbacks=[print_best_callback])
rf_best = rf_study.best_params
rf_best

  0%|          | 0/20 [00:00<?, ?it/s]

Best value: 2674.1177006619255
Best value: 2022.120634903474
Best value: 2022.120634903474
Best value: 2022.120634903474
Best value: 2022.120634903474
Best value: 2022.120634903474
Best value: 2022.120634903474
Best value: 2022.120634903474
Best value: 2022.120634903474
Best value: 2022.120634903474
Best value: 2022.120634903474
Best value: 1593.8798743822397
Best value: 1593.8798743822397
Best value: 1593.8798743822397
Best value: 1593.8798743822397
Best value: 1593.8798743822397
Best value: 1593.8798743822397
Best value: 1593.8798743822397
Best value: 1593.8798743822397
Best value: 1593.8798743822397


{'criterion': 'mae', 'n_estimators': 458, 'max_depth': 7}

In [87]:
#tuned model
rf_model = RandomForestRegressor(**rf_best)

#retrain with full data
rf_tuned_clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', rf_model)])
rf_tuned_clf.fit(X_train, y_train)
rf_tuned_clf.score(X_test, y_test)

0.9073606765803235

## XGBoost

In [88]:
model = XGBRegressor()

# Bundle preprocessing and modeling code in a pipeline
xg_clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model) ])

scores = cross_val_score(xg_clf, X_train, y_train,
                              cv=5,
                              scoring='neg_mean_absolute_error')
print("MAE scores:\n", scores)
print(scores.mean())

MAE scores:
 [ -914.12623352 -1039.79320418 -1312.85442713  -956.28344833
  -772.75798424]
-999.1630594795673


In [89]:
xg_clf.fit(X_train, y_train)
xg_clf.score(X_test, y_test)

0.9309759755301875

### Tuning

In [90]:
def xg_score(params):
    model = XGBRegressor( **params)
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)])
    mae = -1 * cross_val_score(pipe, X_train, y_train, cv=5,
                        scoring='neg_mean_absolute_error').mean()
    return mae


def xg_objective(trial):
    params = {
                "n_estimators" : trial.suggest_int('n_estimators', 0, 500),
                'max_depth':trial.suggest_int('max_depth', 3, 5),
                'reg_alpha':trial.suggest_uniform('reg_alpha',0,6),
                'reg_lambda':trial.suggest_uniform('reg_lambda',0,2),
                'min_child_weight':trial.suggest_int('min_child_weight',0,5),
                'gamma':trial.suggest_uniform('gamma', 0, 4),
                'learning_rate':trial.suggest_loguniform('learning_rate',0.05,0.5),
                'colsample_bytree':trial.suggest_uniform('colsample_bytree',0.4,0.9),
                'subsample':trial.suggest_uniform('subsample',0.4,0.9),
                'nthread' : -1
            }
    return(xg_score(params))


def print_best_callback(study, trial):
    print(f"Best value: {study.best_value}")

In [99]:
optuna.logging.set_verbosity(0)
xg_study = optuna.create_study(direction='minimize',sampler=TPESampler())
xg_study.optimize(xg_objective, n_trials= 50, show_progress_bar = True, callbacks=[print_best_callback])
xg_best = xg_study.best_params
xg_best

  0%|          | 0/50 [00:00<?, ?it/s]

Best value: 5619.2316665824355
Best value: 5576.847202983998
Best value: 2133.0672257443853
Best value: 1427.3289027274932
Best value: 1385.6914986959232
Best value: 1069.4448211120484
Best value: 1069.4448211120484
Best value: 1069.4448211120484
Best value: 1069.4448211120484
Best value: 1069.4448211120484
Best value: 1069.4448211120484
Best value: 1069.4448211120484
Best value: 1069.4448211120484
Best value: 1069.4448211120484
Best value: 1069.4448211120484
Best value: 1069.4448211120484
Best value: 1069.4448211120484
Best value: 1069.4448211120484
Best value: 1069.4448211120484
Best value: 1069.4448211120484
Best value: 1069.4448211120484
Best value: 1054.2802115157706
Best value: 1054.2802115157706
Best value: 1054.2802115157706
Best value: 1054.2802115157706
Best value: 1054.2802115157706
Best value: 1054.2802115157706
Best value: 1054.2802115157706
Best value: 1054.2802115157706
Best value: 1054.2802115157706
Best value: 1054.2802115157706
Best value: 1054.2802115157706
Best valu

{'n_estimators': 447,
 'max_depth': 5,
 'reg_alpha': 1.2659350537942364,
 'reg_lambda': 1.4251114921305825,
 'min_child_weight': 1,
 'gamma': 2.256530635158748,
 'learning_rate': 0.3108450141198587,
 'colsample_bytree': 0.8009936433619437,
 'subsample': 0.5299943085379969}

In [100]:
#tuned model
xg_model = XGBRegressor(**xg_best)

#retrain with full data
xg_tuned_clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', xg_model)])
xg_tuned_clf.fit(X_train, y_train)
xg_tuned_clf.score(X_test, y_test)

0.933489978601395

## Results Analysis

In [101]:
def show_results(actual, pred):
    rsquared = r2_score(actual, pred)
    mae = mean_absolute_error(actual, pred)
    rmse = mean_squared_error(actual, pred, squared=False)
    return pd.DataFrame([rsquared, mae, rmse], index=["$R^2$", "MAE", "RMSE"])

In [102]:
#RF
rf_base_pred = rf_clf.predict(X_test)
rf_tuned_pred = rf_tuned_clf.predict(X_test)

#XG
xg_base_pred = xg_clf.predict(X_test)
xg_tuned_pred = xg_tuned_clf.predict(X_test)

In [103]:
temp = pd.DataFrame()
temp["rf_base"] = show_results(y_test, rf_base_pred)
temp["rf_tuned"] = show_results(y_test, rf_tuned_pred)
temp["xg_base"] = show_results(y_test, xg_base_pred)
temp["xg_tuned"] = show_results(y_test, xg_tuned_pred)
temp.round(4)

,rf_base,rf_tuned,xg_base,xg_tuned
$R^2$,0.9329,0.9074,0.9310,0.9335
MAE,982.1223,1592.0367,979.6887,1026.2704
RMSE,2885.0820,3390.2323,2926.3880,2872.6011


In [104]:
#compare predictions to actual data
temp = pd.DataFrame()
temp["b_rf_pred"] = rf_clf.predict(X_test)
temp["b_xg_pred"] = xg_clf.predict(X_test)
temp["actual"] = y_test.values
temp["rf_pred"] = rf_tuned_clf.predict(X_test)
temp["xg_pred"] = xg_tuned_clf.predict(X_test)
temp.round(3)

,b_rf_pred,b_xg_pred,actual,rf_pred,xg_pred
0,2700.000,2887.912109,2700,2793.450,2761.805908
1,13000.000,13001.898438,13000,11650.016,12977.445312
2,9200.000,9213.899414,9200,9086.195,9629.266602
3,8500.000,8490.784180,8500,7971.921,8498.908203
4,20223.138,20391.218750,3620,12137.822,20503.212891
...,...,...,...,...,...
1068,8927.117,8924.880859,7200,7943.788,8946.518555
1069,13000.000,12970.406250,13000,11663.728,12987.998047
1070,13600.000,13613.195312,13600,12884.498,13490.850586
1071,3644.481,3647.520020,2100,6173.483,3454.766113
